## 概要
* 漢字をカタカナに変換
* MeCabでカタカナに変換できなかったものは除外
* 季語がない無季はとりあえず除外

In [1]:
import MeCab
import pandas as pd
import re
import jaconv

In [2]:
#tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
tagger = MeCab.Tagger("-Ochasen")

In [5]:
new=pd.read_csv("df_cleaning3.csv")

In [7]:
new["kigo"].nunique()

4620

In [39]:
new[new["haiku"].apply(lambda x: "枯蓮" in x)]

,Unnamed: 0,cd-num,haiku,writer,kigo,season,yomi,kigo_yomi
1150,1150,1231,枯蓮の折れたる影は折れてをる,富安風生,枯蓮,冬,枯蓮ノオレタルカゲハオレテヲル,枯蓮
2035,2035,2159,枯蓮のうごく時きてみなうごく,西東三鬼,枯蓮,冬,枯蓮ノウゴクトキキテミナウゴク,枯蓮
2330,2330,2469,枯蓮に隈おとしたる道化たち,橋閒石,枯蓮,冬,枯蓮ニクマオトシタルドウケタチ,枯蓮
6263,6263,7518,枯蓮や無用の者ら無用の首,井沢唯夫,枯蓮,冬,枯蓮ヤムヨウノモノラムヨウノクビ,枯蓮
12015,12015,14283,裏返る力もうなし枯蓮,畠中定子,枯蓮,冬,ウラガエルリキモウナシ枯蓮,枯蓮
12597,12597,14872,杖ついて水の枯蓮歳をとる,竹鼻瑠璃男,枯蓮,冬,ツエツイテミズノ枯蓮トシヲトル,枯蓮
16497,16497,18794,枯蓮己の影の怖からむ,藤岡尚子,枯蓮,冬,枯蓮オノレノカゲノコワカラム,枯蓮
17427,17427,19738,筑波嶺を背に枯蓮枯蓮,菅原けい,枯蓮,冬,ツクバミネヲセニ枯蓮枯蓮,枯蓮
18625,18625,20942,枯蓮に柔らかき水ありにけり,原田宏子,枯蓮,冬,枯蓮ニヤワラカキミズアリニケリ,枯蓮
19440,19440,21764,一望の枯蓮に火の色を見き,江中真弓,枯蓮,冬,イチボウノ枯蓮ニヒノイロヲミキ,枯蓮


In [40]:
def convert(text):
    sentence = []
    if text[-1]=="哉":
        text=text[:-1]+"かな"
    if "枯葎" in text:
        text = re.sub(r"枯葎", "カレムグラ", text)
    if "夏瘦" in text:
        text = re.sub(r"夏瘦", "ナツヤセ", text)
    if "枯薄" in text:
        text = re.sub(r"枯薄", "カレススキ", text)
    if "蟬" in text:
        text = re.sub(r"蟬", "セミ", text)
    if "魂迎" in text:
        text = re.sub(r"魂迎", "タマムカエ", text)
    if "枯蓮" in text:
        text= re.sub(r"枯蓮", "カレハス", text)
    node = tagger.parse(text)
    node = node.split("\n")
    for i in range(len(node)):
        feature = node[i].split("\t")
        if feature[0] == "EOS":
            break
        sentence.append(feature[1])
        #print(feature)
    return jaconv.hira2kata("".join(sentence))

In [41]:
new["yomi"]=new["haiku"].apply(convert)

In [42]:
new["kigo_yomi"]=new["kigo"].apply(convert)

In [44]:
list("あいうおえ")

['あ', 'い', 'う', 'お', 'え']

In [47]:
x = list(map(lambda x: list(x) ,new["yomi"].values))

In [51]:
import numpy as np
vocab=np.unique(np.concatenate(x, axis=0))

In [53]:
for t in vocab:
    print(t)

*
-
0
1
2
3
8
9
?
A
B
C
D
E
G
H
I
K
L
M
N
O
Q
R
S
U
a
c
d
e
f
g
i
j
k
o
s
t
u
―
“
”
…
※
≪
≫
♭
♯
　
、
。
々
〇
〈
〉
《
》
「
」
『
』
ァ
ア
ィ
イ
ゥ
ウ
ェ
エ
ォ
オ
カ
ガ
キ
ギ
ク
グ
ケ
ゲ
コ
ゴ
サ
ザ
シ
ジ
ス
ズ
セ
ゼ
ソ
ゾ
タ
ダ
チ
ヂ
ッ
ツ
ヅ
テ
デ
ト
ド
ナ
ニ
ヌ
ネ
ノ
ハ
バ
パ
ヒ
ビ
ピ
フ
ブ
プ
ヘ
ベ
ペ
ホ
ボ
ポ
マ
ミ
ム
メ
モ
ャ
ヤ
ュ
ユ
ョ
ヨ
ラ
リ
ル
レ
ロ
ヮ
ワ
ヰ
ヱ
ヲ
ン
ヴ
ヶ
・
ー
ヽ
ヾ
一
丁
七
三
下
中
丸
之
乍
乎
乘
九
乞
也
乱
亂
了
予
争
事
二
亍
于
互
五
井
亡
交
京
人
仄
仆
今
仏
仔
代
以
企
会
伝
伸
似
伽
佗
佛
佞
使
來
侫
促
俑
俟
俠
信
俥
修
俯
俳
倒
倖
倚
借
倣
倦
倶
偃
假
偕
側
偸
催
傲
傾
働
像
僞
僧
兄
光
兒
兩
六
其
冬
冱
冲
冶
凉
凍
凜
凡
処
凭
凹
出
刃
切
刎
初
刧
刷
剔
剝
剥
剪
割
助
勁
勃
勿
匂
匐
北
匿
區
千
南
危
卵
原
厦
去
參
友
取
受
叛
口
古
句
叩
叭
可
台
合
吐
吠
吸
吹
吻
吼
呆
呍
呑
味
命
咄
和
咥
咫
咬
咲
咳
哀
唱
唳
啖
啞
啣
啻
啼
喀
喇
喝
喫
單
喰
嗔
嗟
嗤
嘔
嘗
噓
噛
噫
噴
嚏
嚙
嚴
四
囲
圍
園
土
圧
地
坂
坊
埀
埓
域
基
堅
堕
堡
場
塊
塋
塔
塩
塵
墜
墮
墾
壊
壜
壞
士
声
変
夏
夕
外
多
夜
大
夫
夭
奈
奔
奘
奧
女
奴
如
妊
妣
妹
妻
姫
娑
娼
婢
嫩
嬥
嬰
孀
子
孑
孒
孕
字
宥
家
宿
寂
寐
寒
寝
寡
寢
寰
寶
寺
射
將
尊
就
尽
屈
屋
履
山
岐
岩
岸
峠
峭
峽
崙
嵩
嵬
嶮
嶺
巓
巢
巨
已
市
帚
師
帶
年
庁
庭
廚
廢
引
弥
彈
当
彦
彳
径
徑
従
從
微
心
必
忌
忘
忙
応
忰
忽
思
怨
怺
恃
恋
恙
恢
恨
悔
悲
悶
惑
惜
惡
愁
愛
慂
慕
慣
慰
憑
憚
憫
憶
應
懷
懼
戀
戒
戛
截
戯
戰
戲
戾
所
扇
手
打
払
扨
扱
抄
抜


In [31]:
new.shape

(38188, 8)

In [57]:
def is_kana(text):
    if re.fullmatch(r'[ア-ン]*', text):
        return True
    else:
        return False

In [61]:
only_kana = new[new["yomi"].apply(is_kana)]

In [64]:
only_kana.shape

(32475, 8)

In [65]:
only_kana2 = only_kana[only_kana["kigo"]!="無季"]

In [69]:
only_kana2[["yomi", "kigo_yomi"]].to_csv("only_kana2.csv", index=False)

In [70]:
only_kana2.shape

(31623, 8)

In [71]:
only_kana2["kigo_yomi"].nunique()

3985

In [72]:
only_kana2["kigo_yomi"].unique()

array(['キリ', 'カレムグラ', 'ナノハナ', ..., 'ホッキガイ', 'ショシ', 'トシトクジン'], dtype=object)